In [10]:
%load_ext lab_black

In [11]:
import pandas as pd
import requests
import os
import sys
import datetime
import numpy as np
import re
import logging
from dateutil.relativedelta import relativedelta, TH
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import talib as ta

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
ohlcFl = r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\NSEData\ohlcWithDelOI.csv"
yhfinanceFl = r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\NSEData\correctedEODDataYF.csv"

In [12]:
ohlc = pd.read_csv(ohlcFl)
ohlc["date"] = pd.to_datetime(ohlc.date).apply(lambda x: x.date())
yFin = pd.read_csv(yhfinanceFl)
yFin["Date"] = pd.to_datetime(yFin.Date, format="%Y%m%d").apply(lambda x: x.date())

In [13]:
ohlc.loc[ohlc.symbol == "NSE500", "date"].max()

datetime.date(2022, 4, 6)

In [14]:
yFin.rename(
    columns={
        "Date": "date",
        "Ticker": "symbol",
        "Close": "corrClose",
        "High": "corrHigh",
        "Low": "corrLow",
        "Open": "corrOpen",
        "Volume": "yfVolume",
    },
    inplace=True,
)

In [15]:
finalOHLC = pd.merge(
    left=yFin,
    right=ohlc,
    left_on=["symbol", "date"],
    right_on=["symbol", "date"],
    how="outer",
)

In [16]:
def getNifty500():

    nList = "nifty500"
    url = "https://archives.nseindia.com/content/indices/ind_" + nList + "list.csv"

    return pd.read_csv(url)


nifty500 = getNifty500()

In [17]:
tenYearAgo = datetime.date.today() - relativedelta(years=10)
finalOHLC10yr = finalOHLC.loc[finalOHLC.date >= tenYearAgo, :]

In [18]:
finalOHLC10yrNifty500 = finalOHLC10yr.loc[
    finalOHLC10yr.symbol.isin(nifty500.Symbol), :
].dropna(subset=["corrOpen", "corrClose", "corrHigh", "corrLow"])

In [19]:
finalOHLC10yrNifty500 = finalOHLC10yrNifty500[
    [
        "date",
        "symbol",
        "corrClose",
        "corrHigh",
        "corrLow",
        "corrOpen",
        "volume",
        "delivery",
        "OI",
        "company",
        "industry",
    ]
]
finalOHLC10yrNifty500.rename(
    columns={
        "corrClose": "close",
        "corrHigh": "high",
        "corrLow": "low",
        "corrOpen": "open",
    },
    inplace=True,
)
finalOHLC10yrNifty500.head()

,date,symbol,close,high,low,open,volume,delivery,OI,company,industry
840194,2012-04-09,3MINDIA,4176.450195,4200.000000,3950.000000,3950.050049,526.0,315.0,NaN,3M India Ltd.,Diversified
840195,2012-04-09,AARTIDRUGS,4.442236,4.500094,4.313662,4.500094,1645.0,1196.0,NaN,Aarti Drugs Ltd.,Healthcare
840196,2012-04-09,AARTIIND,12.758076,13.005805,12.500024,12.613566,20737.0,15713.0,NaN,Aarti Industries Ltd.,Chemicals
840197,2012-04-09,ABB,769.768311,785.003810,766.476693,780.818741,41517.0,19613.0,204250.0,ABB India Ltd.,Capital Goods
840198,2012-04-09,ABBOTINDIA,1430.964478,1457.422193,1419.017223,1434.122891,786.0,654.0,NaN,Abbott India Ltd.,Healthcare


In [20]:
finalOHLC10yrNifty500.drop_duplicates(subset=["symbol", "date"], inplace=True)

In [21]:
finalOHLC10yrIndex = finalOHLC10yr.loc[
    finalOHLC10yr.industry == "INDEX",
    [
        "date",
        "symbol",
        "open",
        "high",
        "low",
        "close",
        "volume",
        "delivery",
        "OI",
        "company",
        "industry",
    ],
]
finalOHLC10yrIndex.head()

,date,symbol,open,high,low,close,volume,delivery,OI,company,industry
1841926,2012-04-09,NSENIFTY,5282.50,5287.90,5228.00,5234.40,111914936.0,0.0,19537200.0,IND_NSENIFTY,INDEX
1841927,2012-04-10,NSENIFTY,5254.10,5255.80,5211.85,5243.60,156106794.0,0.0,19279650.0,IND_NSENIFTY,INDEX
1841928,2012-04-11,NSENIFTY,5209.45,5263.65,5190.80,5226.85,167013527.0,0.0,19449050.0,IND_NSENIFTY,INDEX
1841929,2012-04-12,NSENIFTY,5246.75,5290.60,5246.75,5276.85,152684788.0,0.0,NaN,IND_NSENIFTY,INDEX
1841930,2012-04-13,NSENIFTY,5255.70,5306.75,5185.40,5207.45,164225858.0,0.0,NaN,IND_NSENIFTY,INDEX


In [22]:
finalOHLC10yrIndex.drop_duplicates(subset=["symbol", "date"], inplace=True)

In [23]:
finalOHLC10yrIndex.loc[finalOHLC10yrIndex.symbol == "NSE500", "date"].max()

datetime.date(2022, 4, 6)

In [24]:
othersImp = [
    "CRUDE_OIL",
    "GOLD",
    "HDFC_US_ADR",
    "ICICI_US_ADR",
    "INFOSYS_US_ADR",
    "USDINR",
    "DR_REDDY_US_ADR",
    "SILVER",
    "TATAMOTORS_US_ADR",
    "WIPRO_US_ADR",
]
finalOHLC10yrOther = finalOHLC10yr.loc[
    finalOHLC10yr.symbol.isin(othersImp),
    [
        "date",
        "symbol",
        "corrClose",
        "corrHigh",
        "corrLow",
        "corrOpen",
        "yfVolume",
        "delivery",
        "OI",
        "company",
        "industry",
    ],
]
finalOHLC10yrOther["company"] = "OTHERS"
finalOHLC10yrOther["industry"] = "OTHERS"
finalOHLC10yrOther.head()

,date,symbol,corrClose,corrHigh,corrLow,corrOpen,yfVolume,delivery,OI,company,industry
29661,2012-04-06,USDINR,51.033001,51.000000,51.000000,51.000000,0.0,NaN,NaN,OTHERS,OTHERS
29662,2012-04-09,CRUDE_OIL,102.459999,102.599998,100.809998,102.529999,205924.0,NaN,NaN,OTHERS,OTHERS
29663,2012-04-09,GOLD,1642.500000,1646.400024,1636.500000,1645.300049,158.0,NaN,NaN,OTHERS,OTHERS
29664,2012-04-09,HDFC_US_ADR,15.652812,15.713282,15.545822,15.676070,887600.0,NaN,NaN,OTHERS,OTHERS
29665,2012-04-09,ICICI_US_ADR,5.436715,5.465455,5.380832,5.430328,5603400.0,NaN,NaN,OTHERS,OTHERS


In [25]:
finalOHLC10yrOther.drop_duplicates(subset=["symbol", "date"], inplace=True)

In [26]:
finalOHLC10yrOther.rename(
    columns={
        "corrClose": "close",
        "corrHigh": "high",
        "corrLow": "low",
        "corrOpen": "open",
        "yfVolume": "volume",
    },
    inplace=True,
)

In [28]:
corrDf = pd.DataFrame(columns=["stock", "testStock", "corr"])
for sym in finalOHLC10yrNifty500.symbol.unique():
    industry = finalOHLC10yrNifty500.loc[
        finalOHLC10yrNifty500.symbol == sym, "industry"
    ].values[0]
    stock = finalOHLC10yrNifty500.loc[
        finalOHLC10yrNifty500.symbol == sym, ["date", "symbol", "close"]
    ].sort_values(by=["date"])
    stock["ret"] = stock["close"].pct_change()
    stockNumOfDates = stock.date.shape[0]
    for testSym in finalOHLC10yrNifty500.loc[
        (
            (finalOHLC10yrNifty500.industry == industry)
            & (finalOHLC10yrNifty500.symbol != sym)
        ),
        :,
    ]["symbol"].unique():
        testStock = finalOHLC10yrNifty500.loc[
            finalOHLC10yrNifty500.symbol == testSym, ["date", "symbol", "close"]
        ].sort_values(by=["date"])
        noOfDates = testStock.shape[0]
        if stockNumOfDates == noOfDates:
            testStock["ret"] = testStock["close"].pct_change()
            corr, _ = pearsonr(
                stock["ret"].dropna().values, testStock["ret"].dropna().values
            )
            corrDf = corrDf.append(
                pd.DataFrame({"stock": [sym], "testStock": [testSym], "corr": [corr]})
            )
        else:
            commonDates = np.intersect1d(testStock.date, stock.date)

            stock1 = stock.loc[
                stock.date.isin(commonDates), ["date", "symbol", "close"]
            ]
            testStock1 = testStock.loc[
                testStock.date.isin(commonDates), ["date", "symbol", "close"]
            ]

            stock1["ret"] = stock1["close"].pct_change()
            testStock1["ret"] = testStock1["close"].pct_change()
            corr, _ = pearsonr(
                stock1["ret"].dropna().values, testStock1["ret"].dropna().values
            )
            corrDf = corrDf.append(
                pd.DataFrame({"stock": [sym], "testStock": [testSym], "corr": [corr]})
            )

    for testSym in finalOHLC10yrIndex.symbol.unique():
        testStock = finalOHLC10yrIndex.loc[
            finalOHLC10yrIndex.symbol == testSym, ["date", "symbol", "close"]
        ].sort_values(by=["date"])
        testNoOfDates = testStock.shape[0]
        if stockNumOfDates == testNoOfDates:
            testStock["ret"] = testStock["close"].pct_change()
            corr, _ = pearsonr(
                stock["ret"].dropna().values, testStock["ret"].dropna().values
            )
            corrDf = corrDf.append(
                pd.DataFrame({"stock": [sym], "testStock": [testSym], "corr": [corr]})
            )
        else:
            commonDates = np.intersect1d(testStock.date, stock.date)

            stock1 = stock.loc[
                stock.date.isin(commonDates), ["date", "symbol", "close"]
            ]
            testStock1 = testStock.loc[
                testStock.date.isin(commonDates), ["date", "symbol", "close"]
            ]

            stock1["ret"] = stock1["close"].pct_change()
            testStock1["ret"] = testStock1["close"].pct_change()
            corr, _ = pearsonr(
                stock1["ret"].dropna().values, testStock1["ret"].dropna().values
            )
            corrDf = corrDf.append(
                pd.DataFrame({"stock": [sym], "testStock": [testSym], "corr": [corr]})
            )
    for testSym in finalOHLC10yrOther.symbol.unique():
        testStock = finalOHLC10yrOther.loc[
            finalOHLC10yrOther.symbol == testSym, ["date", "symbol", "close"]
        ].sort_values(by=["date"])
        testNoOfDates = testStock.shape[0]
        if stockNumOfDates == testNoOfDates:
            testStock["ret"] = testStock["close"].pct_change()
            corr, _ = pearsonr(
                stock["ret"].dropna().values, testStock["ret"].dropna().values
            )
            corrDf = corrDf.append(
                pd.DataFrame({"stock": [sym], "testStock": [testSym], "corr": [corr]})
            )
        else:
            commonDates = np.intersect1d(testStock.date, stock.date)

            stock1 = stock.loc[
                stock.date.isin(commonDates), ["date", "symbol", "close"]
            ]
            testStock1 = testStock.loc[
                testStock.date.isin(commonDates), ["date", "symbol", "close"]
            ]

            stock1["ret"] = stock1["close"].pct_change()
            testStock1["ret"] = testStock1["close"].pct_change()

            if (
                stock1["ret"].dropna().shape[0] > 3
                and testStock1["ret"].dropna().shape[0] > 3
            ):
                try:
                    corr, _ = pearsonr(
                        stock1["ret"].dropna().values, testStock1["ret"].dropna().values
                    )
                    corrDf = corrDf.append(
                        pd.DataFrame(
                            {"stock": [sym], "testStock": [testSym], "corr": [corr]}
                        )
                    )
                except:
                    continue

    # break

ValueError: x and y must have length at least 2.

In [21]:
corrDf.sort_values(by=["corr"], ascending=False).to_csv(
    r"/Users/shubhangi/DeepDocuments/Trading/NSEData/corr.csv", index=False
)

NameError: name 'corrDf' is not defined

In [27]:
allData = finalOHLC10yrOther.append(finalOHLC10yrIndex).append(finalOHLC10yrNifty500)
allData["date"] = allData.date.apply(lambda x: x.strftime("%Y%m%d"))
allData.fillna(0, inplace=True)
allData.to_csv(
    r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\NSEData\nifty500OtherOHLC10yrs.csv",
    index=False,
)

In [28]:
allData.loc[allData.symbol == "TCS", "date"].max()

'20220406'

### Below code is for analysis stock by stock

In [34]:
symbol = "CHALET"

In [35]:
import nsepy as npy
import datetime
from dateutil.relativedelta import relativedelta, TH
import re

start_date = datetime.date.today() - relativedelta(months=6)
end_date = datetime.date.today()

ohlc_data = npy.get_history(
    symbol=re.sub("&", "%26", symbol), start=start_date, end=end_date
).reset_index()

In [36]:
ohlc_data["vol_per_trade"] = ohlc_data["Volume"] / ohlc_data["Trades"]
ohlc_data.sort_values(by="Date", ascending=False).to_csv(
    r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\NSEData\ohlc_data.csv",
    index=False,
)

In [32]:
ohlc_data.head()

,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble,vol_per_trade
0,2021-10-04,ZYDUSWELL,EQ,2320.95,2339.00,2351.40,2301.00,2311.0,2315.75,2330.54,18960,4.418695e+12,3682,9471,0.4995,5.149375
1,2021-10-05,ZYDUSWELL,EQ,2315.75,2327.80,2375.00,2306.05,2340.0,2342.60,2349.33,317237,7.452949e+13,5269,289334,0.9120,60.208199
2,2021-10-06,ZYDUSWELL,EQ,2342.60,2350.00,2364.80,2310.05,2318.0,2320.00,2347.40,158427,3.718915e+13,5414,141261,0.8916,29.262468
3,2021-10-07,ZYDUSWELL,EQ,2320.00,2335.45,2335.45,2301.10,2307.0,2309.15,2318.10,18450,4.276903e+12,3116,10597,0.5744,5.921053
4,2021-10-08,ZYDUSWELL,EQ,2309.15,2324.40,2395.65,2301.00,2375.0,2366.75,2352.69,43842,1.031466e+13,3964,26439,0.6031,11.060040


In [42]:
# symbol = symbol

In [43]:
mypath = r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data"
holidays = pd.read_excel(
    mypath + os.path.sep + "NSEData" + os.path.sep + "nse_holidays.xlsx"
)
holidays.holidays = holidays.holidays.apply(lambda x: x.date())
# symbols = pd.read_csv(mypath + os.path.sep + "NSEData" + os.path.sep + "FNO.csv")
today = datetime.datetime.today()
from_date = today - relativedelta(months=6)
dtList = list(pd.date_range(from_date, today, freq="B").to_pydatetime())
expriyDf = pd.DataFrame()
oiData = pd.DataFrame()
for dd in dtList:
    if holidays.holidays.isin([datetime.date(dd.year, dd.month, dd.day)]).sum() == 0:
        end_of_month = dd + relativedelta(day=31)
        expiry_date = end_of_month + relativedelta(weekday=TH(-1))
        while True:
            if (
                holidays.holidays.isin(
                    [
                        datetime.date(
                            expiry_date.year, expiry_date.month, expiry_date.day
                        )
                    ]
                ).sum()
                == 0
            ):
                break
            else:
                expiry_date = expiry_date - relativedelta(day=1)
        end_of_next_month = dd + relativedelta(months=1, day=31)
        expiry_date_next_month = end_of_next_month + relativedelta(weekday=TH(-1))
        while True:
            if (
                holidays.holidays.isin(
                    [
                        datetime.date(
                            expiry_date_next_month.year,
                            expiry_date_next_month.month,
                            expiry_date_next_month.day,
                        )
                    ]
                ).sum()
                == 0
            ):
                break
            else:
                expiry_date_next_month = expiry_date_next_month - relativedelta(day=1)

        end_of_month_after_next_month = dd + relativedelta(months=2, day=31)
        expiry_date_month_after_next_month = (
            end_of_month_after_next_month + relativedelta(weekday=TH(-1))
        )
        while True:
            if (
                holidays.holidays.isin(
                    [
                        datetime.date(
                            expiry_date_month_after_next_month.year,
                            expiry_date_month_after_next_month.month,
                            expiry_date_month_after_next_month.day,
                        )
                    ]
                ).sum()
                == 0
            ):
                break
            else:
                expiry_date_month_after_next_month = (
                    expiry_date_month_after_next_month - relativedelta(day=1)
                )
        if dd > expiry_date:
            expiry_date = expiry_date_next_month
            expiry_date_next_month = expiry_date_month_after_next_month
            end_of_month_after_next_month = dd + relativedelta(months=3, day=31)
            expiry_date_month_after_next_month = (
                end_of_month_after_next_month + relativedelta(weekday=TH(-1))
            )
            while True:
                if (
                    holidays.holidays.isin(
                        [
                            datetime.date(
                                expiry_date_month_after_next_month.year,
                                expiry_date_month_after_next_month.month,
                                expiry_date_month_after_next_month.day,
                            )
                        ]
                    ).sum()
                    == 0
                ):
                    break
                else:
                    expiry_date_month_after_next_month = (
                        expiry_date_month_after_next_month - relativedelta(day=1)
                    )

        expriyDf = expriyDf.append(
            pd.DataFrame(
                {
                    "date": [dd],
                    "current_exp_date": [expiry_date],
                    "next_month_exp_date": [expiry_date_next_month],
                    "month_after_next_month_exp_date": [
                        expiry_date_month_after_next_month
                    ],
                }
            )
        )
for _, row in expriyDf.iterrows():
    logging.info(
        "fetching features data for %s for date %s exp dates %s %s %s"
        % (
            symbol,
            datetime.datetime.strftime(row["date"].date(), "%Y-%m-%d"),
            datetime.datetime.strftime(row["current_exp_date"].date(), "%Y-%m-%d"),
            datetime.datetime.strftime(row["next_month_exp_date"].date(), "%Y-%m-%d"),
            datetime.datetime.strftime(
                row["month_after_next_month_exp_date"].date(), "%Y-%m-%d"
            ),
        )
    )
    current_month_features = npy.get_history(
        symbol=symbol,
        start=row["date"].date(),
        end=row["date"].date(),
        futures=True,
        expiry_date=row["current_exp_date"].date(),
    )
    next_month_features = npy.get_history(
        symbol=symbol,
        start=row["date"].date(),
        end=row["date"].date(),
        futures=True,
        expiry_date=row["next_month_exp_date"].date(),
    )
    month_after_next_month_features = npy.get_history(
        symbol=symbol,
        start=row["date"].date(),
        end=row["date"].date(),
        futures=True,
        expiry_date=row["month_after_next_month_exp_date"].date(),
    )
    if (
        (len(current_month_features["Open Interest"].values) == 1)
        and (len(next_month_features["Open Interest"].values) > 0)
        and (len(month_after_next_month_features["Open Interest"].values) > 0)
    ):
        oiData = oiData.append(
            pd.DataFrame(
                {
                    "Symbol": [symbol],
                    "Date": [row["date"].date()],
                    "cummOI": [
                        current_month_features["Open Interest"].values[0]
                        + next_month_features["Open Interest"].values[0]
                        + month_after_next_month_features["Open Interest"].values[0]
                    ],
                }
            )
        )

In [44]:
ohlc_oi_date = pd.merge(
    left=oiData,
    right=ohlc_data,
    left_on=["Date", "Symbol"],
    right_on=["Date", "Symbol"],
    how="inner",
)

In [45]:
# ohlc_oi_date["vol_per_trade"] = ohlc_oi_date["Volume"] / ohlc_oi_date["Trades"]
ohlc_oi_date.sort_values(by="Date", ascending=False).to_csv(
    r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\NSEData\ohlc_oi_date.csv",
    index=False,
)

In [46]:
ohlc_oi_date.set_index("Date", inplace=True)
ohlc_oi_date.sort_index(inplace=True)

In [47]:
ohlc_oi_date["5 Day avg Del Vol"] = ta.MA(
    ohlc_oi_date["Deliverable Volume"], 5, ta.MA_Type.SMA
).shift(1)

In [48]:
avgDelPct = ohlc_oi_date["%Deliverble"].mean()
avgVolPerTrade = ohlc_oi_date["vol_per_trade"].mean()

In [49]:
ohlc_oi_date["OI Chng"] = ohlc_oi_date["cummOI"] - ohlc_oi_date["cummOI"].shift(1)
ohlc_oi_date["% OI Change"] = (
    ohlc_oi_date["cummOI"] - ohlc_oi_date["cummOI"].shift(1)
) / ohlc_oi_date["cummOI"].shift(1)
ohlc_oi_date["% Price Change"] = (
    ohlc_oi_date["Close"] - ohlc_oi_date["Close"].shift(1)
) / ohlc_oi_date["Close"].shift(1)

In [50]:
def oiAnalysis(prcChng, oiChg):
    if prcChng > 0 and oiChg > 0:
        return "LongBuildUp"
    elif prcChng > 0 and oiChg < 0:
        return "ShortCovering"
    elif prcChng < 0 and oiChg > 0:
        return "ShortBuildUp"
    elif prcChng < 0 and oiChg < 0:
        return "LongUnWinding"


ohlc_oi_date["oiAnalysis"] = ohlc_oi_date.apply(
    lambda x: oiAnalysis(x["% Price Change"], x["% OI Change"]), axis=1
)

In [51]:
ohlc_oi_date.sort_index(ascending=False, inplace=True)

In [52]:
last5Days = 5
LBD_Days = sum(ohlc_oi_date.iloc[0:last5Days, :]["oiAnalysis"] == "LongBuildUp")
cls_higher_vwap = ohlc_oi_date.iloc[0, :]["Close"] > ohlc_oi_date.iloc[0, :]["VWAP"]
del_pct_higher_than_avg = ohlc_oi_date.iloc[0, :]["%Deliverble"] > avgDelPct
vol_per_trade_than_avg = ohlc_oi_date.iloc[0, :]["vol_per_trade"] > avgVolPerTrade

In [53]:
signal = "None"
if (
    del_pct_higher_than_avg
    and vol_per_trade_than_avg
    and cls_higher_vwap
    and LBD_Days > 1
    and ohlc_oi_date.iloc[0, :]["oiAnalysis"] == "LongBuildUp"
):
    signal = "VERY_STRONG"
elif (
    del_pct_higher_than_avg
    and vol_per_trade_than_avg
    and cls_higher_vwap
    and LBD_Days == 1
    and ohlc_oi_date.iloc[0, :]["oiAnalysis"] == "LongBuildUp"
):
    signal = "STRONG"
else:
    signal = "None"
print(signal)

None


### download FII data

In [ ]:
endDate = datetime.date.today()
startDate = endDate - relativedelta(months=6)
dtSrs = pd.Series(pd.date_range(startDate, endDate, freq="SM")).apply(
    lambda x: x.strftime("%b%d%Y")
)
dtSrsLong = pd.Series(pd.date_range(startDate, endDate, freq="SM")).apply(
    lambda x: x.strftime("%B%d%Y")
)
dateDf = pd.DataFrame({"shrtDt": dtSrs, "longDt": dtSrsLong})
data = pd.DataFrame()

for _, dd in dateDf.iterrows():
    urlShrt = (
        f"https://www.fpi.nsdl.co.in/web/StaticReports/Fortnightly_Sector_wise_FII_Investment_Data/FIIInvestSector_%s.html"
        % (dd["shrtDt"])
    )
    urlLng = (
        f"https://www.fpi.nsdl.co.in/web/StaticReports/Fortnightly_Sector_wise_FII_Investment_Data/FIIInvestSector_%s.html"
        % (dd["longDt"])
    )
    print(urlShrt)
    if data.shape[0] == 0:
        try:
            data = pd.read_html(urlShrt)[0].iloc[3:, [1, 32]]
            data.columns = ["Sector", dd["shrtDt"]]

        except:
            data = pd.read_html(urlLng)[0].iloc[3:, [1, 32]]
            data.columns = ["Sector", dd["longDt"]]

    else:
        try:
            temp = pd.read_html(urlShrt)[0].iloc[3:, [1, 32]]
            temp.columns = ["Sector", dd["shrtDt"]]
        except:
            temp = pd.read_html(urlLng)[0].iloc[3:, [1, 32]]
            temp.columns = ["Sector", dd["longDt"]]
        data = pd.merge(
            left=data, right=temp, left_on="Sector", right_on="Sector", how="outer"
        )

data.to_csv(mypath + os.path.sep + "NSEData" + os.path.sep + "fii.csv", index=False)

### Find big move stocks


In [29]:
import pandas as pd
import requests
import os
import sys
import datetime
import numpy as np
import re
import logging
from dateutil.relativedelta import relativedelta, TH
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import talib as ta

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
import nsepy as npy

logging.basicConfig(
    format="Date-Time : %(asctime)s : Line No. : %(lineno)d - %(message)s",
    level=logging.INFO,
)
monthsBack = 6


def oiAnalysis(prcChng, oiChg):
    if prcChng > 0 and oiChg > 0:
        return "LongBuildUp"
    elif prcChng > 0 and oiChg < 0:
        return "ShortCovering"
    elif prcChng < 0 and oiChg > 0:
        return "ShortBuildUp"
    elif prcChng < 0 and oiChg < 0:
        return "LongUnWinding"


analysis = pd.DataFrame(columns=["Symbol", "Signal"])
print("numpy version %s" % np.__version__)
start_date = datetime.date.today() - relativedelta(months=monthsBack)
end_date = datetime.date.today()

mypath = r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data"

holidays = pd.read_excel(
    mypath + os.path.sep + "NSEData" + os.path.sep + "nse_holidays.xlsx"
)
holidays.holidays = holidays.holidays.apply(lambda x: x.date())
symbols = pd.read_csv(mypath + os.path.sep + "NSEData" + os.path.sep + "FNO.csv")
today = end_date
from_date = start_date
dtList = list(pd.date_range(from_date, today, freq="B").to_pydatetime())
expriyDf = pd.DataFrame()

for dd in dtList:
    if holidays.holidays.isin([datetime.date(dd.year, dd.month, dd.day)]).sum() == 0:
        end_of_month = dd + relativedelta(day=31)
        expiry_date = end_of_month + relativedelta(weekday=TH(-1))
        while True:
            if (
                holidays.holidays.isin(
                    [
                        datetime.date(
                            expiry_date.year, expiry_date.month, expiry_date.day
                        )
                    ]
                ).sum()
                == 0
            ):
                break
            else:
                expiry_date = expiry_date - relativedelta(day=1)
        end_of_next_month = dd + relativedelta(months=1, day=31)
        expiry_date_next_month = end_of_next_month + relativedelta(weekday=TH(-1))
        while True:
            if (
                holidays.holidays.isin(
                    [
                        datetime.date(
                            expiry_date_next_month.year,
                            expiry_date_next_month.month,
                            expiry_date_next_month.day,
                        )
                    ]
                ).sum()
                == 0
            ):
                break
            else:
                expiry_date_next_month = expiry_date_next_month - relativedelta(day=1)

        end_of_month_after_next_month = dd + relativedelta(months=2, day=31)
        expiry_date_month_after_next_month = (
            end_of_month_after_next_month + relativedelta(weekday=TH(-1))
        )
        while True:
            if (
                holidays.holidays.isin(
                    [
                        datetime.date(
                            expiry_date_month_after_next_month.year,
                            expiry_date_month_after_next_month.month,
                            expiry_date_month_after_next_month.day,
                        )
                    ]
                ).sum()
                == 0
            ):
                break
            else:
                expiry_date_month_after_next_month = (
                    expiry_date_month_after_next_month - relativedelta(day=1)
                )
        if dd > expiry_date:
            expiry_date = expiry_date_next_month
            expiry_date_next_month = expiry_date_month_after_next_month
            end_of_month_after_next_month = dd + relativedelta(months=3, day=31)
            expiry_date_month_after_next_month = (
                end_of_month_after_next_month + relativedelta(weekday=TH(-1))
            )
            while True:
                if (
                    holidays.holidays.isin(
                        [
                            datetime.date(
                                expiry_date_month_after_next_month.year,
                                expiry_date_month_after_next_month.month,
                                expiry_date_month_after_next_month.day,
                            )
                        ]
                    ).sum()
                    == 0
                ):
                    break
                else:
                    expiry_date_month_after_next_month = (
                        expiry_date_month_after_next_month - relativedelta(day=1)
                    )

        expriyDf = expriyDf.append(
            pd.DataFrame(
                {
                    "date": [dd],
                    "current_exp_date": [expiry_date],
                    "next_month_exp_date": [expiry_date_next_month],
                    "month_after_next_month_exp_date": [
                        expiry_date_month_after_next_month
                    ],
                }
            )
        )

numpy version 1.20.1


In [30]:
allOI_data = pd.DataFrame()
for symbol in pd.read_csv(
    r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\WatchList\fno_list.tls",
    header=None,
).values:
    # if symbol[0].strip()=='TCS' :
    try:
        try:
            ohlc_oi_date_existing = pd.read_csv(
                r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\NSEData\OHLC_VWAP_OI\ohlc_oi_date_"
                + symbol[0]
                + ".csv"
            )
            ohlc_oi_date_existing.Date = ohlc_oi_date_existing.Date.apply(
                lambda x: datetime.datetime.strptime(x, "%Y-%m-%d")
            )
            ohlc_oi_date_existing.set_index("Date", inplace=True)
            existing_end_date = ohlc_oi_date_existing.index.date.max() + relativedelta(
                days=1
            )
        except:
            ohlc_oi_date_existing = pd.DataFrame()
            existing_end_date = start_date
        # existing_end_date = datetime.date(2022,3,30)
        if existing_end_date <= end_date:
            ohlc_data = npy.get_history(
                symbol=re.sub("&", "%26", symbol[0]),
                start=existing_end_date,
                end=end_date,
            ).reset_index()
        else:
            ohlc_data = pd.DataFrame()
        if ohlc_data.shape[0] >= 1:

            ohlc_data["vol_per_trade"] = ohlc_data["Volume"] / ohlc_data["Trades"]
            oiData = pd.DataFrame()
            expriyDf = expriyDf.loc[
                expriyDf.date.apply(lambda x: x.date()) >= existing_end_date, :
            ]

            for _, row in expriyDf.iterrows():
                logging.info(
                    "run for %s  date %s exp date %s"
                    % (
                        symbol[0],
                        datetime.date.strftime(row["date"].date(), "%Y-%m-%d"),
                        datetime.date.strftime(
                            row["current_exp_date"].date(), "%Y-%m-%d"
                        ),
                    )
                )
                current_month_features = npy.get_history(
                    symbol=re.sub("&", "%26", symbol[0]),
                    start=row["date"].date(),
                    end=row["date"].date(),
                    futures=True,
                    expiry_date=row["current_exp_date"].date(),
                )
                next_month_features = npy.get_history(
                    symbol=re.sub("&", "%26", symbol[0]),
                    start=row["date"].date(),
                    end=row["date"].date(),
                    futures=True,
                    expiry_date=row["next_month_exp_date"].date(),
                )
                month_after_next_month_features = npy.get_history(
                    symbol=re.sub("&", "%26", symbol[0]),
                    start=row["date"].date(),
                    end=row["date"].date(),
                    futures=True,
                    expiry_date=row["month_after_next_month_exp_date"].date(),
                )
                # print(current_month_features.shape)
                if (
                    (len(current_month_features["Open Interest"].values) == 1)
                    and (len(next_month_features["Open Interest"].values) > 0)
                    and (
                        len(month_after_next_month_features["Open Interest"].values) > 0
                    )
                ):
                    oiData = oiData.append(
                        pd.DataFrame(
                            {
                                "Symbol": [symbol[0]],
                                "Date": [row["date"].date()],
                                "cummOI": [
                                    current_month_features["Open Interest"].values[0]
                                    + next_month_features["Open Interest"].values[0]
                                    + month_after_next_month_features[
                                        "Open Interest"
                                    ].values[0]
                                ],
                            }
                        )
                    )
            if oiData.shape[0] >= 1:
                ohlc_oi_date = pd.merge(
                    left=oiData,
                    right=ohlc_data,
                    left_on=["Date", "Symbol"],
                    right_on=["Date", "Symbol"],
                    how="inner",
                )
            if ohlc_oi_date_existing.shape[0] > 0:
                ohlc_oi_date_existing = (
                    ohlc_oi_date_existing.loc[
                        (
                            (ohlc_oi_date_existing.index.date < existing_end_date)
                            & (ohlc_oi_date_existing.index.date >= start_date)
                        ),
                        :,
                    ]
                    .reset_index()
                    .loc[:, list(ohlc_oi_date.columns)]
                )

                ohlc_oi_date = ohlc_oi_date.append(ohlc_oi_date_existing)
            ohlc_oi_date.set_index("Date", inplace=True)
            ohlc_oi_date.sort_index(inplace=True)

            ohlc_oi_date["5 Day avg Close Price"] = ta.EMA(
                ohlc_oi_date["Close"], 5
            ).shift(1)
            ohlc_oi_date["13 Day avg Close"] = ta.EMA(ohlc_oi_date["Close"], 13).shift(
                1
            )
            ohlc_oi_date["21 Day avg Close"] = ta.EMA(ohlc_oi_date["Close"], 21).shift(
                1
            )

            ohlc_oi_date["5 Day avg Del Vol"] = ta.MA(
                ohlc_oi_date["Deliverable Volume"], 5, ta.MA_Type.SMA
            ).shift(1)
            ohlc_oi_date["13 Day avg Del Vol"] = ta.MA(
                ohlc_oi_date["Deliverable Volume"], 13, ta.MA_Type.SMA
            ).shift(1)
            ohlc_oi_date["21 Day avg Del Vol"] = ta.MA(
                ohlc_oi_date["Deliverable Volume"], 21, ta.MA_Type.SMA
            ).shift(1)
            ohlc_oi_date["5 Day avg vol_per_trade"] = ta.MA(
                ohlc_oi_date["vol_per_trade"], 5, ta.MA_Type.SMA
            ).shift(1)
            ohlc_oi_date["13 Day avg vol_per_trade"] = ta.MA(
                ohlc_oi_date["vol_per_trade"], 13, ta.MA_Type.SMA
            ).shift(1)
            ohlc_oi_date["21 Day avg vol_per_trade"] = ta.MA(
                ohlc_oi_date["vol_per_trade"], 21, ta.MA_Type.SMA
            ).shift(1)

            ohlc_oi_date["isHAMMER"] = (
                ta.CDLHAMMER(
                    ohlc_oi_date.Open,
                    ohlc_oi_date.High,
                    ohlc_oi_date.Low,
                    ohlc_oi_date.Close,
                )
                == 100
            )
            ohlc_oi_date["isHARAMI"] = (
                ta.CDLHARAMI(
                    ohlc_oi_date.Open,
                    ohlc_oi_date.High,
                    ohlc_oi_date.Low,
                    ohlc_oi_date.Close,
                )
                == 100
            )

            avgDelPct = ohlc_oi_date["%Deliverble"].mean()
            avgVolPerTrade = ohlc_oi_date["vol_per_trade"].mean()
            ohlc_oi_date["OI Chng"] = ohlc_oi_date["cummOI"] - ohlc_oi_date[
                "cummOI"
            ].shift(1)
            ohlc_oi_date["% OI Change"] = (
                ohlc_oi_date["cummOI"] - ohlc_oi_date["cummOI"].shift(1)
            ) / ohlc_oi_date["cummOI"].shift(1)
            ohlc_oi_date["% Price Change"] = (
                ohlc_oi_date["Close"] - ohlc_oi_date["Close"].shift(1)
            ) / ohlc_oi_date["Close"].shift(1)
            ohlc_oi_date["oiAnalysis"] = ohlc_oi_date.apply(
                lambda x: oiAnalysis(x["% Price Change"], x["% OI Change"]), axis=1
            )
            ohlc_oi_date.sort_index(ascending=False, inplace=True)
            last5Days = 5
            LBD_Days = sum(
                ohlc_oi_date.iloc[0:last5Days, :]["oiAnalysis"] == "LongBuildUp"
            )
            cls_higher_vwap = (
                ohlc_oi_date.iloc[0, :]["Close"] > ohlc_oi_date.iloc[0, :]["VWAP"]
            )
            del_pct_higher_than_avg = ohlc_oi_date.iloc[0, :]["%Deliverble"] > avgDelPct
            vol_per_trade_than_avg = (
                ohlc_oi_date.iloc[0, :]["vol_per_trade"] > avgVolPerTrade
            )
            signal = "None"
            if (
                del_pct_higher_than_avg
                and vol_per_trade_than_avg
                and cls_higher_vwap
                and LBD_Days > 1
                and ohlc_oi_date.iloc[0, :]["oiAnalysis"] == "LongBuildUp"
            ):
                signal = "VERY_STRONG"
            elif (
                del_pct_higher_than_avg
                and vol_per_trade_than_avg
                and cls_higher_vwap
                and ohlc_oi_date.iloc[0, :]["oiAnalysis"] == "LongBuildUp"
            ):
                signal = "STRONG"
            elif (
                del_pct_higher_than_avg
                and vol_per_trade_than_avg
                and LBD_Days > 1
                and ohlc_oi_date.iloc[0, :]["oiAnalysis"] == "LongBuildUp"
            ):
                signal = "STRONG_NO_VWAP"
            elif (
                cls_higher_vwap
                and LBD_Days > 1
                and ohlc_oi_date.iloc[0, :]["oiAnalysis"] == "LongBuildUp"
            ):
                signal = "STRONG_NO_DEL"
            else:
                signal = ohlc_oi_date.iloc[0, :]["oiAnalysis"]
            analysis = analysis.append(
                pd.DataFrame({"Symbol": [symbol[0]], "Signal": [signal]})
            )
            ohlc_oi_date = ohlc_oi_date.sort_index(ascending=False).reset_index()
            ohlc_oi_date["Date"] = ohlc_oi_date["Date"].apply(
                lambda x: datetime.datetime.strftime(x, "%Y-%m-%d")
            )
            ohlc_oi_date = ohlc_oi_date.set_index("Date")
            ohlc_oi_date.to_csv(
                r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\NSEData\OHLC_VWAP_OI\ohlc_oi_date_"
                + symbol[0]
                + ".csv"
            )

            allOI_data = allOI_data.append(ohlc_oi_date)
            analysis.to_csv(
                r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\NSEData\bigMoveAnalysis.csv",
                index=False,
            )
            logging.info("Symbol %s analysis %s" % (symbol[0], signal))
        elif ohlc_oi_date_existing.shape[0] > 0:
            allOI_data = allOI_data.append(ohlc_oi_date_existing)
    except:
        logging.error("error ")
allOI_data.to_csv(
    r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\NSEData\allOI_data_vwap.csv"
)

Date-Time : 2022-04-06 20:55:49,281 : Line No. : 42 - run for AARTIIND  date 2022-04-06 exp date 2022-04-28
Date-Time : 2022-04-06 20:55:49,937 : Line No. : 245 - Symbol AARTIIND analysis ShortBuildUp
Date-Time : 2022-04-06 20:55:50,217 : Line No. : 42 - run for ABBOTINDIA  date 2022-04-06 exp date 2022-04-28
Date-Time : 2022-04-06 20:55:50,803 : Line No. : 245 - Symbol ABBOTINDIA analysis LongBuildUp
Date-Time : 2022-04-06 20:55:51,087 : Line No. : 42 - run for ABCAPITAL  date 2022-04-06 exp date 2022-04-28
Date-Time : 2022-04-06 20:55:52,477 : Line No. : 245 - Symbol ABCAPITAL analysis STRONG_NO_DEL
Date-Time : 2022-04-06 20:55:52,777 : Line No. : 42 - run for ABFRL  date 2022-04-06 exp date 2022-04-28
Date-Time : 2022-04-06 20:55:53,361 : Line No. : 245 - Symbol ABFRL analysis STRONG_NO_DEL
Date-Time : 2022-04-06 20:55:53,601 : Line No. : 42 - run for ACC  date 2022-04-06 exp date 2022-04-28
Date-Time : 2022-04-06 20:55:54,210 : Line No. : 245 - Symbol ACC analysis ShortBuildUp
Date

In [14]:
logging.info(
    "run for %s  date %s exp date %s"
    % (
        symbols[0],
        datetime.date.strftime(row["date"].date(), "%Y-%m-%d"),
        datetime.date.strftime(row["current_exp_date"].date(), "%Y-%m-%d"),
    )
)


KeyError: 0